In [1]:
import numpy as np
import matplotlib as plt
import time
from tqdm import *


In [2]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

In [3]:
input_size = 4
hidden_size = 4
batch_size = 1
idx2char = ['e','h','l','o']
x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2]

one_hot_lookup = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]
]

x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(-1,batch_size,input_size)
#inputs.shape(seqlen,batch_size,input_size)
labels = torch.LongTensor(y_data).view(-1,1)
#labels.size(seq.size,1)
print(inputs)
print(labels)
print(labels.shape)
x = zip(inputs,labels)
print(list(x))

tensor([[[0., 1., 0., 0.]],

        [[1., 0., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 1.]]])
tensor([[3],
        [1],
        [2],
        [3],
        [2]])
torch.Size([5, 1])
[(tensor([[0., 1., 0., 0.]]), tensor([3])), (tensor([[1., 0., 0., 0.]]), tensor([1])), (tensor([[0., 0., 1., 0.]]), tensor([2])), (tensor([[0., 0., 1., 0.]]), tensor([3])), (tensor([[0., 0., 0., 1.]]), tensor([2]))]


In [4]:
class Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size):
        super(Model,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.rnncell = torch.nn.RNNCell(input_size=self.input_size,hidden_size=self.hidden_size)
        #实例化RNNcell
    def forward(self,input,hidden):
        hidden = self.rnncell(input,hidden)
        return hidden

    def init_hidden(self):
        return torch.zeros(self.batch_size,self.hidden_size)
    #如果在函数外就设置了h0，则省略这一步，并且不再需要batch_size
    #hidden.shape(numLayers,batch_size,hidden_size)
net = Model(input_size,hidden_size,batch_size)

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.1)

In [6]:
for epoch in range(15):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden() #设置初始h0
    print('predicted:',end = '')
    for input ,label in zip(inputs,labels):
        #zip将合并两个Tensor，其中的data一一对应
        #input.size(batch_size,input_size)
        #label.size(1)
        hidden = net(input,hidden)
        #向前传播
        #inputs.shape(seqlen,batch_size,input_size)
        #hidden.shape(numLayers,batch_size,hidden_size)
        loss += criterion(hidden,label)
        #网络的输出直接输入不需要激活函数，label也不需要变成one_hot向量
        #序列中的每一步的损失都累加，即同一个cell的损失要累加
        _,idx = hidden.max(dim=1)
        print(idx2char[idx.item()],end='')
    loss.backward()
    optimizer.step()
    print(',Epoch[%d/15] loss = %.4f' % (epoch+1,loss.item()))

predicted:hhhhh,Epoch[1/15] loss = 8.4057
predicted:hhhho,Epoch[2/15] loss = 6.8713
predicted:ohlll,Epoch[3/15] loss = 5.6024
predicted:ohlol,Epoch[4/15] loss = 4.7212
predicted:ohlol,Epoch[5/15] loss = 4.0734
predicted:ohlol,Epoch[6/15] loss = 3.5519
predicted:ohlol,Epoch[7/15] loss = 3.1731
predicted:ohlol,Epoch[8/15] loss = 2.9017
predicted:ohlol,Epoch[9/15] loss = 2.6959
predicted:ohlol,Epoch[10/15] loss = 2.5220
predicted:ohlol,Epoch[11/15] loss = 2.3679
predicted:ohlol,Epoch[12/15] loss = 2.2432
predicted:ohlol,Epoch[13/15] loss = 2.1553
predicted:ohlol,Epoch[14/15] loss = 2.0940
predicted:ohlol,Epoch[15/15] loss = 2.0433
